In [1]:
!pip install mxnet
!pip install dgl

     |████████████████████████████████| 46.9 MB 40 kB/s 
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1
     |████████████████████████████████| 4.4 MB 4.9 MB/s 


In [2]:
# Need to set the dgl backend
import os

os.environ["DGLBACKEND"] = "mxnet"

In [3]:
import dgl.data
dataset = dgl.data.CoraGraphDataset()


Using backend: mxnet


Extracting file to /root/.dgl/cora_v2
Finished data loading and preprocessing.
  NumNodes: 2708
  NumEdges: 10556
  NumFeats: 1433
  NumClasses: 7
  NumTrainingSamples: 140
  NumValidationSamples: 500
  NumTestSamples: 1000
Done saving data into cached files.


In [4]:
g = dataset[0]

In [5]:
print('Node features')
print(g.ndata)
print('Edge features')
print(g.edata)

Node features
{'train_mask': 
[1. 1. 1. ... 0. 0. 0.]
<NDArray 2708 @cpu(0)>, 'val_mask': 
[0. 0. 0. ... 0. 0. 0.]
<NDArray 2708 @cpu(0)>, 'test_mask': 
[0. 0. 0. ... 1. 1. 1.]
<NDArray 2708 @cpu(0)>, 'label': 
[3 4 4 ... 3 3 3]
<NDArray 2708 @cpu(0)>, 'feat': 
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
<NDArray 2708x1433 @cpu(0)>}
Edge features
{}


In [70]:
from dgl.nn import GraphConv
from mxnet import gluon, init, npx, autograd
from mxnet.gluon import nn, rnn
import mxnet as mx

class GCN(nn.Block):
    def __init__(self, in_feats, h_feats, num_classes, **kwargs):
        super().__init__(**kwargs)
        self.conv1 = GraphConv(in_feats, h_feats)
        self.activ = nn.Activation("relu")
        self.conv2 = GraphConv(h_feats, num_classes)
        self.sig = nn.Activation("sigmoid")

    def forward(self, g, in_feat):
        h = self.conv1(g, in_feat)
        h = self.activ(h)
        h = self.conv2(g, h)
        #h = self.sig(h)
        return h

net = GCN(g.ndata['feat'].shape[1], 16, dataset.num_classes)
net.initialize(init.Normal(0.01))

In [7]:
dataset.num_classes

7

In [71]:
# [TO-DO] training function
def train(g, net, n_epochs, train_params):
    trainer = gluon.Trainer(net.collect_params(), "adam", train_params)
    loss_func = gluon.loss.SoftmaxCrossEntropyLoss()

    features = g.ndata['feat']
    labels = g.ndata['label']
    labels_one_hot = mx.nd.one_hot(labels, 7)
    train_mask_bool = list(g.ndata['train_mask'].asnumpy())
    valid_mask_bool = list(g.ndata['val_mask'].asnumpy())
    test_mask_bool = list(g.ndata['test_mask'].asnumpy())

    train_mask = []
    valid_mask = []
    test_mask = []
    for item in [(train_mask_bool, train_mask), (valid_mask_bool, valid_mask), (test_mask_bool, test_mask)]:
        for i in range(len(item[0])):
            if item[0][i]==1: item[1].append(int(i))

    train_mask = mx.ndarray.ndarray.array(train_mask, dtype="int32") 
    valid_mask = mx.ndarray.ndarray.array(valid_mask, dtype="int32") 
    test_mask = mx.ndarray.ndarray.array(test_mask, dtype="int32")
    #train_mask = mx.ndarray.ndarray.array(train_mask) 
    #valid_mask = mx.ndarray.ndarray.array(valid_mask) 
    #test_mask = mx.ndarray.ndarray.array(test_mask)
    
    for epoch in range(1, n_epochs+1):
        with autograd.record():
            y_hat = net(g, features)
            loss = loss_func(y_hat[train_mask], labels[train_mask])
            #print(loss.shape)

        loss.backward()
        trainer.step(1)

        pred = y_hat.argmax(1)
        pred = mx.nd.array(pred, dtype="int64")
        #print(pred[train_mask]-labels[train_mask])
        #print(pred[valid_mask]-labels[valid_mask])
        
        #print(pred[valid_mask])
        #print(labels[valid_mask])
        #print(float(mx.nd.sum(pred[valid_mask]==labels[valid_mask]).asnumpy()))
        #print(len(pred[valid_mask]))
        train_acc = float(mx.nd.sum(pred[train_mask]==labels[train_mask]).asnumpy())/len(pred[train_mask])
        valid_acc = float(mx.nd.sum(pred[valid_mask]==labels[valid_mask]).asnumpy())/len(pred[valid_mask])
        test_acc = float(mx.nd.sum(pred[test_mask]==labels[test_mask]).asnumpy())/len(pred[test_mask])
        print(f"epoch {epoch}: {float(train_acc):f} / {float(valid_acc):f} / {float(test_acc):f} / {float(loss.mean().asnumpy()):f}")

train(g, net, 100, {"learning_rate" : 0.01})

epoch 1: 0.128571 / 0.150000 / 0.115000 / 1.945959
epoch 2: 0.214286 / 0.114000 / 0.131000 / 1.942345
epoch 3: 0.492857 / 0.318000 / 0.328000 / 1.937484
epoch 4: 0.628571 / 0.442000 / 0.489000 / 1.930079
epoch 5: 0.785714 / 0.508000 / 0.506000 / 1.921127
epoch 6: 0.785714 / 0.502000 / 0.486000 / 1.912048
epoch 7: 0.771429 / 0.508000 / 0.487000 / 1.902626
epoch 8: 0.792857 / 0.522000 / 0.511000 / 1.891933
epoch 9: 0.814286 / 0.528000 / 0.539000 / 1.879992
epoch 10: 0.821429 / 0.536000 / 0.552000 / 1.867304
epoch 11: 0.842857 / 0.560000 / 0.557000 / 1.853449
epoch 12: 0.842857 / 0.574000 / 0.569000 / 1.838529
epoch 13: 0.835714 / 0.578000 / 0.577000 / 1.822859
epoch 14: 0.850000 / 0.578000 / 0.587000 / 1.806265
epoch 15: 0.864286 / 0.596000 / 0.597000 / 1.788679
epoch 16: 0.850000 / 0.610000 / 0.608000 / 1.770063
epoch 17: 0.864286 / 0.618000 / 0.616000 / 1.750437
epoch 18: 0.878571 / 0.626000 / 0.625000 / 1.729845
epoch 19: 0.885714 / 0.628000 / 0.632000 / 1.708329
epoch 20: 0.900000 / 

In [ ]:
labels = g.ndata['label']
labels_one_hot = mx.nd.one_hot(labels, 7)
mx.nd.argmax(mx.nd.abs(net(g, g.ndata['feat'])-labels_one_hot))


[1207.]
<NDArray 1 @cpu(0)>

In [ ]:
mx.nd.sum(mx.nd.abs(net(g, g.ndata['feat'])-labels_one_hot))


[1343.6486]
<NDArray 1 @cpu(0)>

In [ ]:
len(labels_one_hot)

2708

In [ ]:
net(g, g.ndata['feat'])[1207]


[7.4344091e-05 1.1085745e-08 4.6792929e-06 1.0696469e-09 7.8606173e-02
 2.8811020e-10 1.8635015e-05]
<NDArray 7 @cpu(0)>

In [ ]:
labels_one_hot[1207]


[0. 0. 0. 0. 1. 0. 0.]
<NDArray 7 @cpu(0)>

In [ ]:
type(g.ndata['train_mask'])

mxnet.ndarray.ndarray.NDArray

In [ ]:
g.ndata['train_mask'].shape

(2708,)

In [ ]:
mx.nd.sum(g.ndata['train_mask'])[0]


[140.]
<NDArray 1 @cpu(0)>